In [ ]:
import os  # used for directory operations
import tensorflow as tf
from PIL import Image  # used to read images from directory
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# path to record files
cwd = Path('..')
# path to folder to save tfrecords
recordPath = Path('..')

In [ ]:
## functions creating features of certain types required to write a tfrecord
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    # If the value is an eager tensor BytesList won't unpack a string from an EagerTensor.
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() 
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
BATCH_SIZE = 64
IMAGE_SIZE = [224, 224]

In [ ]:
keys = []
values = []
for folder in os.listdir(cwd):
    keys.append(folder)
for i in range(0,len(keys)):
    values.append(i)
classes = dict(zip(keys, values))

In [ ]:
print(classes)

In [ ]:
filenames = os.listdir(recordPath)
for i in  range(len(filenames)):
    filenames[i] = recordPath + filenames[i]
print(filenames)

In [ ]:
feature_description = {
    'image': tf.io.FixedLenFeature((), tf.string),
    'label': tf.io.FixedLenFeature((), tf.int64),
    'height': tf.io.FixedLenFeature((), tf.int64),
    'width': tf.io.FixedLenFeature((), tf.int64),
    'depth': tf.io.FixedLenFeature((), tf.int64)
}
def _parse_image_function(example_proto):
  # Parse the input tf.train.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, feature_description)

In [ ]:
def read_records(filename,num_pics,num_records):
  ## num_pics - pictures to read from single records
  ## num_records - number of records to read from dir (if more than one)
  for n, record in enumerate(filename):
    tfrecord_dataset = tf.data.TFRecordDataset(record)
    parsed_image_dataset = tfrecord_dataset.map(_parse_image_function)
    if n == num_records:
      break
    for i,image_features in enumerate(parsed_image_dataset):
        if i == num_pics:
          break
        img_raw = tf.io.parse_tensor(image_features['image'],tf.uint8)
        label = image_features['label'].numpy()
        image_shape = [image_features['height'].numpy(), image_features['width'].numpy(), image_features['depth'].numpy()]
        image = tf.reshape(img_raw, image_shape)
        image = image.numpy()
  
        print(image.shape)
        plt.imshow(image)
        plt.title(list(classes.keys())[list(classes.values()).index(label)])
        plt.show()
       
    


In [ ]:
read_records(filenames,5,7)